### Создание эмбеддингов для аудиодорожек, взятых из видео-файлов с помощью модели YAMNET

[ссылка на модель](https://www.kaggle.com/models/google/yamnet)


In [19]:
import subprocess
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sn
import torch
import torchaudio
from IPython import display
from IPython.display import clear_output
from sklearn.metrics import confusion_matrix
from torch import nn
from torch.utils.data import DataLoader, Dataset, Subset
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import csv
import os
import matplotlib.pyplot as plt
from IPython.display import Audio
from scipy.io import wavfile
import scipy
!apt-get install ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


**Дальше идет процесс извлечения и получения эмбеддинга аудио**

In [47]:
def extract_audio_from_video(video_file, output_audio):
  # Извлечение аудио из видео с помощью FFmpeg
  command = f"ffmpeg -i {video_file} -q:a 0 -map a -ar 16000 {output_audio} -y"
  subprocess.call(command, shell=True)

In [28]:
def ensure_sample_rate(original_sample_rate, waveform,
                       desired_sample_rate=16000):
  """Resample waveform if required."""
  if original_sample_rate != desired_sample_rate:
    desired_length = int(round(float(len(waveform)) /
                               original_sample_rate * desired_sample_rate))
    waveform = scipy.signal.resample(waveform, desired_length)
  return desired_sample_rate, waveform

In [29]:
# Find the name of the class with the top score when mean-aggregated across frames.
def class_names_from_csv(class_map_csv_text):
  """Returns list of class names corresponding to score vector."""
  class_names = []
  with tf.io.gfile.GFile(class_map_csv_text) as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
      class_names.append(row['display_name'])

  return class_names

#### Создание эмбеддинга при помощи модели

In [33]:
# Загрузка модели YAMNet
model = hub.load('https://tfhub.dev/google/yamnet/1')

# Получение карты классов для модели
class_map_path = model.class_map_path().numpy()
class_names = class_names_from_csv(class_map_path)

video_folder = 'extracted_files/videos_2/'
video_ids = []
audio_logits = np.array([])
for video_file in os.listdir(video_folder):
  output_audio = 'extracted_files/audios_full/' + video_file[:-3] + 'wav' # создаем новый путь для аудио
  full_video_path = 'extracted_files/videos_2/' + video_file
  extract_audio_from_video(full_video_path, output_audio)

  sample_rate, wav_data = wavfile.read(output_audio, 'rb')

  # Преобразование стерео в моно, если необходимо
  if wav_data.ndim > 1:
    wav_data = np.mean(wav_data, axis=1)
  # Приведение к нужной частоте дискретизации
  sample_rate, wav_data = ensure_sample_rate(sample_rate, wav_data)

  waveform = wav_data / tf.int16.max

  # Прогон через модель
  scores, embeddings, spectrogram = model(waveform)

  # Получение предсказаний
  scores_np = scores.numpy()
  video_ids.append(video_file)
  audio_logits = np.append(scores_np.mean(axis=0), audio_logits, axis = 0) # список эмбеддингов, то что как раз и нужно было на входе из аудиодорожки, shape = 1 x 521, вероятности 521 класса

Создаю csv-file который дальше передается куда нужно...

In [40]:
data_to_extract = pd.DataFrame(audio_logits.reshape(-1, 521))
data_to_extract['video_ids'] = video_ids

In [42]:
with open('audio_logits_to_extract.csv', 'w') as file:
    for index, row in data_to_extract.iterrows():
        line = ''
        for column in data_to_extract.columns:
            line += f'{column}={row[column]}\t'
        print(line.strip(), end='\n', file=file)